In [1]:
%env CUDA_VISIBLE_DEVICES="0"
import sys
sys.path.append('..')

env: CUDA_VISIBLE_DEVICES="0"


In [5]:
import datetime as dt
from pathlib import Path

import IPython.display as ipd
import numpy as np
import soundfile as sf
import torch
from tqdm.auto import tqdm

from pflow.models.pflow_tts import pflowTTS
from pflow.text import sequence_to_text, text_to_sequence
from pflow.utils.model import denormalize
from pflow.utils.utils import get_user_data_dir, intersperse

from pflow.hifigan.config import v1
from pflow.hifigan.denoiser import Denoiser
from pflow.hifigan.env import AttrDict
from pflow.hifigan.models import Generator as HiFiGAN

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
# This allows for real time code changes being reflected in the notebook, no need to restart the kernel

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
device = "cpu"

In [6]:
get_user_data_dir()

PosixPath('/home/user/.local/share/pflow_tts')

## Filepaths

In [17]:
PFLOW_CHECKPOINT = "/home/user/Documents/AlexBiloshytskyi/TTS/pflowtts_pytorch/logs/train/ljspeech/runs/2024-04-09_15-46-42/checkpoints/last.ckpt" #fill in the path to the pflow checkpoint
HIFIGAN_CHECKPOINT = "../pflow/hifigan/pretrained/generator_v1" #get_user_data_dir() / "pflow/hifigan/pretrained/generator_v1"
OUTPUT_FOLDER = "synth_output"

## Load TTS model

In [8]:
def load_model(checkpoint_path):
    model = pflowTTS.load_from_checkpoint(checkpoint_path, map_location=device)
    model.eval()
    return model
count_params = lambda x: f"{sum(p.numel() for p in x.parameters()):,}"


model = load_model(PFLOW_CHECKPOINT)
print(f"Model loaded! Parameter count: {count_params(model)}")

Model loaded! Parameter count: 18,827,505


In [18]:
def load_vocoder(checkpoint_path):
    h = AttrDict(v1)
    hifigan = HiFiGAN(h).to(device)
    hifigan.load_state_dict(torch.load(checkpoint_path, map_location=device)['generator'])
    _ = hifigan.eval()
    hifigan.remove_weight_norm()
    return hifigan

vocoder = load_vocoder(HIFIGAN_CHECKPOINT)
denoiser = Denoiser(vocoder, mode='zeros')

/home/user/Documents/AlexBiloshytskyi/TTS/venv_nlp/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Removing weight norm...


### Helper functions to synthesise

In [19]:
@torch.inference_mode()
def process_text(text: str):
    x = torch.tensor(intersperse(text_to_sequence(text, ['english_cleaners2']), 0),dtype=torch.long, device=device)[None]
    x_lengths = torch.tensor([x.shape[-1]],dtype=torch.long, device=device)
    x_phones = sequence_to_text(x.squeeze(0).tolist())
    return {
        'x_orig': text,
        'x': x,
        'x_lengths': x_lengths,
        'x_phones': x_phones
    }


@torch.inference_mode()
def synthesise(text, prompt):
    text_processed = process_text(text)
    start_t = dt.datetime.now()
    output = model.synthesise(
        text_processed['x'], 
        text_processed['x_lengths'],
        n_timesteps=n_timesteps,
        temperature=temperature,
        length_scale=length_scale,
        prompt=prompt
    )
    # merge everything to one dict    
    output.update({'start_t': start_t, **text_processed})
    return output

@torch.inference_mode()
def to_waveform(mel, vocoder):
    audio = vocoder(mel).clamp(-1, 1)
    audio = denoiser(audio.squeeze(0), strength=0.00025).cpu().squeeze()
    return audio.cpu().squeeze()
    
# def save_to_folder(filename: str, output: dict, folder: str):
#     folder = Path(folder)
#     folder.mkdir(exist_ok=True, parents=True)
#     np.save(folder / f'{filename}', output['mel'].cpu().numpy())
#     sf.write(folder / f'{filename}.wav', output['waveform'], 22050, 'PCM_24')

## Setup text to synthesise

In [20]:
texts = ["Setup text to synthesise"

]

### Hyperparameters

In [21]:
## Number of ODE Solver steps
n_timesteps = 10

## Changes to the speaking rate
length_scale=1.0

## Sampling temperature
temperature = 0.667

In [23]:
import torchaudio
import glob
path_ljspeech = "../data/LJSpeech-1.1/wavs/*.wav"
wav_files = glob.glob(path_ljspeech) ## fill in the path to the LJSpeech-1.1 dataset
wav, sr = torchaudio.load(wav_files[0])
from pflow.data.text_mel_datamodule import mel_spectrogram
mel = mel_spectrogram(
            wav,
            1024,
            80,
            22050,
            256,
            1024,
            0,
            8000,
            center=False,
        )

In [24]:
prompt = mel #load a mel spectrogram from a file and paste it here; check dimensions [batch, channels, time]

In [26]:
prompt.shape

torch.Size([1, 80, 676])

## Synthesis

In [27]:
outputs, rtfs = [], []
rtfs_w = []
for i, text in enumerate(tqdm(texts)):
    prompt = prompt[:,:,:264]
    from pflow.utils.model import normalize
    prompt = normalize(prompt, model.mel_mean, model.mel_std)
    output = synthesise(text, prompt) #, torch.tensor([15], device=device, dtype=torch.long).unsqueeze(0))
    output['waveform'] = to_waveform(output['mel'], vocoder)

    # Compute Real Time Factor (RTF) with HiFi-GAN
    t = (dt.datetime.now() - output['start_t']).total_seconds()
    rtf_w = t * 22050 / (output['waveform'].shape[-1])

    ## Pretty print
    print(f"{'*' * 53}")
    print(f"Input text - {i}")
    print(f"{'-' * 53}")
    print(output['x_orig'])
    print(f"{'*' * 53}")
    print(f"Phonetised text - {i}")
    print(f"{'-' * 53}")
    print(output['x_phones'])
    print(f"{'*' * 53}")
    print(f"RTF:\t\t{output['rtf']:.6f}")
    print(f"RTF Waveform:\t{rtf_w:.6f}")
    rtfs.append(output['rtf'])
    rtfs_w.append(rtf_w)

    # Display the synthesised waveform
    ipd.display(ipd.Audio(output['waveform'], rate=22050))

    ## Save the generated waveform
#     save_to_folder(i, output, OUTPUT_FOLDER)

print(f"Number of ODE steps: {n_timesteps}")
print(f"Mean RTF:\t\t\t\t{np.mean(rtfs):.6f} ± {np.std(rtfs):.6f}")
print(f"Mean RTF Waveform (incl. vocoder):\t{np.mean(rtfs_w):.6f} ± {np.std(rtfs_w):.6f}")

  0%|          | 0/1 [00:00<?, ?it/s]

*****************************************************
Input text - 0
-----------------------------------------------------
Setup text to synthesise
*****************************************************
Phonetised text - 0
-----------------------------------------------------
_s_ˈ_ɛ_ɾ_ʌ_p_ _t_ˈ_ɛ_k_s_t_ _t_ə_ _s_ˈ_ɪ_n_θ_ə_s_ˌ_a_ɪ_z_
*****************************************************
RTF:		0.104604
RTF Waveform:	0.362344


Number of ODE steps: 10
Mean RTF:				0.104604 ± 0.000000
Mean RTF Waveform (incl. vocoder):	0.362344 ± 0.000000
